<a href="https://colab.research.google.com/github/aryannewyork/BlockFed/blob/master/FL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setting Up Libraries and Packages

In [ ]:
!pip install kora -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 44.0 MB/s eta 0:00:00


In [ ]:
from kora import drive
drive.link_nbs()

Mounted at /content/drive


In [ ]:
from flutils import *

importing Jupyter notebook from /nbs/flutils.ipynb


In [ ]:
import numpy as np
import random
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer 
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from tqdm import tqdm
import time

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers.legacy import SGD
from tensorflow.keras import backend as K

## Loading and Processing Dataset

In [ ]:
data_path = '/content/drive/MyDrive/Datasets/swarm_aligned'

In [ ]:
#apply our function
data_list, label_list = load(data_path)
labels = list(set(label_list.tolist())) #unique labels

#binarize the labels
#lb = LabelBinarizer()
#label_list = lb.fit_transform(label_list)
n_values = np.max(label_list) + 1
label_list = np.eye(n_values)[label_list]

In [ ]:
#split data into training and test set
X_train, X_test, y_train, y_test = train_test_split(data_list, 
                                                    label_list, 
                                                    test_size=0.1, 
                                                    random_state=42)

## Creating Clients to Participate in FL

In [ ]:
#create clients
clients = create_clients(X_train, y_train, num_clients=10, initial='client')

In [ ]:
#process and batch the training data for each client
clients_batched = dict()
for (client_name, data) in clients.items():
    clients_batched[client_name] = batch_data(data)
    
#process and batch the test set  
test_batched = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(len(y_test))

comms_round = 10 #number of global epochs
    

## Creating Optimizer (To Perform Gradient Descent)

In [ ]:
#create optimizer
lr = 0.01 
loss='categorical_crossentropy'
metrics = ['accuracy']
optimizer = SGD(lr=lr, 
                decay=lr / comms_round, 
                momentum=0.9
               ) 

#initialize global model
#print(data_list.shape,labels)
smlp_global = SimpleMLP()
global_model = smlp_global.build(data_list.shape[1],len(labels))

/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


## Training and Averaging

In [ ]:
#commence global training loop
for comm_round in range(comms_round):
            
    # get the global model's weights - will serve as the initial weights for all local models
    global_weights = global_model.get_weights()
    
    #initial list to collect local model weights after scalling
    scaled_local_weight_list = list()

    #randomize client data - using keys
    client_names= list(clients_batched.keys())
    random.shuffle(client_names)
    
    #loop through each client and create new local model
    for client in tqdm(client_names , desc = 'Progress Bar'):
        #time.sleep(0.5)
        smlp_local = SimpleMLP()
        local_model = smlp_local.build(data_list.shape[1],len(labels))
        local_model.compile(loss=loss, 
                      optimizer=optimizer, 
                      metrics=metrics)
        
        #print(local_model.summary())
        #print(clients_batched)
        #set local model weight to the weight of the global model
        local_model.set_weights(global_weights)
        
        #fit local model with client's data
        local_model.fit(clients_batched[client], epochs=1, verbose=0)
        
        #scale the model weights and add to list
        scaling_factor = weight_scalling_factor(clients_batched, client)
        scaled_weights = scale_model_weights(local_model.get_weights(), scaling_factor)
        scaled_local_weight_list.append(scaled_weights)
        
        #clear session to free memory after each communication round
        K.clear_session()
        
    lobal_acc: 69.734%
    
    #update global model 
    global_model.set_weights(average_weights)

    #test global model and print out metrics after each communications round
    for(X_test, Y_test) in test_batched:
        global_acc, global_loss = test_model(X_test, Y_test, global_model, comm_round)


Progress Bar: 100%|██████████| 10/10 [00:20<00:00,  2.04s/it]


76/76 [==============================] - 1s 5ms/step
comm_round: 0 | global_acc: 69.734% | global_loss: 0.6601933240890503


Progress Bar: 100%|██████████| 10/10 [00:16<00:00,  1.66s/it]


76/76 [==============================] - 0s 6ms/step
comm_round: 1 | global_acc: 69.734% | global_loss: 0.6595581769943237


Progress Bar: 100%|██████████| 10/10 [00:22<00:00,  2.28s/it]


...

Progress Bar: 100%|██████████| 10/10 [00:18<00:00,  1.90s/it]


76/76 [==============================] - 0s 5ms/step
comm_round: 6 | global_acc: 69.734% | global_loss: 0.6596036553382874


Progress Bar: 100%|██████████| 10/10 [00:19<00:00,  1.99s/it]


76/76 [==============================] - 0s 5ms/step
comm_round: 7 | global_acc: 69.734% | global_loss: 0.6597053408622742


Progress Bar: 100%|██████████| 10/10 [00:17<00:00,  1.79s/it]

In [ ]:
SGD_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(len(y_train)).batch(320)
smlp_SGD = SimpleMLP()
SGD_model = smlp_SGD.build(data_list.shape[1], len(labels)) 

SGD_model.compile(loss=loss, 
              optimizer=optimizer, 
              metrics=metrics)

In [ ]:
# fit the SGD training data to model
_ = SGD_model.fit(SGD_dataset, epochs=100, verbose=0)

In [ ]:
#test the SGD global model and print out metrics
for(X_test, Y_test) in test_batched:
        SGD_acc, SGD_loss = test_model(X_test, Y_test, SGD_model, 1)

76/76 [==============================] - 1s 7ms/step
comm_round: 1 | global_acc: 69.734% | global_loss: 0.6598128080368042
